## Step 1 : Create a simple tensor with random items

In [1]:
import numpy as np

# Suppress scientific notation
np.set_printoptions(suppress=True)

# Generate randomly distributed parameters
params = np.random.uniform(low=-50, high=150, size=20)

# Make sure important values are at the beginning for better debugging
params[0] = params.max() + 1
params[1] = params.min() - 1
params[2] = 0

# Round each number to the second decimal place
params = np.round(params, 2)

# Print the parameters
print(params)

[120.7  -28.11   0.    57.36  69.31 118.77  83.29  15.46 -27.11  87.43
  92.15 109.84  93.99  41.57  48.05 119.7  -20.97 -10.14 -19.4   15.34]


## Step 2 : Define the quantization methods and quantize

In [2]:
def clamp(params_q: np.array, lower_bound: int, upper_bound: int) -> np.array:
    params_q[params_q < lower_bound] = lower_bound
    params_q[params_q > upper_bound] = upper_bound
    return params_q

def asymmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float, int]:
    # Calculate the scale and zero point
    alpha                    = np.max(params)
    beta                     = np.min(params)
    scale                    = (alpha - beta) / (2**bits-1)
    zero                     = -1*np.round(beta / scale)
    lower_bound, upper_bound = 0, 2**bits-1
    # Quantize the parameters
    quantized                = clamp(np.round(params / scale + zero), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero

def asymmetric_dequantize(params_q: np.array, scale: float, zero: int) -> np.array:
    return (params_q - zero) * scale

def symmetric_dequantize(params_q: np.array, scale: float) -> np.array:
    return params_q * scale

def symmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float]:
    # Calculate the scale
    alpha       = np.max(np.abs(params))
    scale       = alpha / (2**(bits-1)-1)
    lower_bound = -2**(bits-1)
    upper_bound = 2**(bits-1)-1
    # Quantize the parameters
    quantized   = clamp(np.round(params / scale), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale

def quantization_error(params: np.array, params_q: np.array):
    # calculate the MSE
    return np.mean((params - params_q)**2)

In [3]:
(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale)                    = symmetric_quantization(params, 8)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}')
print(asymmetric_q)
print('')
print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)

Original:
[120.7  -28.11   0.    57.36  69.31 118.77  83.29  15.46 -27.11  87.43
  92.15 109.84  93.99  41.57  48.05 119.7  -20.97 -10.14 -19.4   15.34]

Asymmetric scale: 0.5835686274509804, zero: 48.0
[255   0  48 146 167 252 191  74   2 198 206 236 209 119 130 253  12  31
  15  74]

Symmetric scale: 0.9503937007874016
[127 -30   0  60  73 125  88  16 -29  92  97 116  99  44  51 126 -22 -11
 -20  16]


## Step 3 : Dequantize them back

In [4]:
# Dequantize the parameters back to 32 bits
params_deq_asymmetric = asymmetric_dequantize(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric  = symmetric_dequantize(symmetric_q, symmetric_scale)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Dequantize Asymmetric:')
print(np.round(params_deq_asymmetric,2))
print('')
print(f'Dequantize Symmetric:')
print(np.round(params_deq_symmetric, 2))

Original:
[120.7  -28.11   0.    57.36  69.31 118.77  83.29  15.46 -27.11  87.43
  92.15 109.84  93.99  41.57  48.05 119.7  -20.97 -10.14 -19.4   15.34]

Dequantize Asymmetric:
[120.8  -28.01   0.    57.19  69.44 119.05  83.45  15.17 -26.84  87.54
  92.2  109.71  93.95  41.43  47.85 119.63 -21.01  -9.92 -19.26  15.17]

Dequantize Symmetric:
[120.7  -28.51   0.    57.02  69.38 118.8   83.63  15.21 -27.56  87.44
  92.19 110.25  94.09  41.82  48.47 119.75 -20.91 -10.45 -19.01  15.21]


## Step 4 : Quantization error

In [5]:
# Calculate the quantization error
print(f'{"Asymmetric error: ":>20}{np.round(quantization_error(params, params_deq_asymmetric), 2)}')
print(f'{"Symmetric error: ":>20}{np.round(quantization_error(params, params_deq_symmetric), 2)}')

  Asymmetric error: 0.03
   Symmetric error: 0.07
